In this section, we’ll present a step-by-step guide on fine-tuning an encoder-only transformer model for music classification. We’ll use a lightweight model for this demonstration and fairly small dataset, meaning the code is runnable end-to-end on any consumer grade GPU, including the T4 16GB GPU provided in the Google Colab free tier. The section includes various tips that you can try should you have a smaller GPU and encounter memory issues along the way.



## The Dataset

To train our model, we’ll use the GTZAN dataset, which is a popular dataset of 1,000 songs for music genre classification. Each song is a 30-second clip from one of 10 genres of music, spanning disco to metal. We can get the audio files and their corresponding labels from the Hugging Face Hub with the load_dataset() function from 🤗 Datasets:



In [1]:
!pip install -q datasets[audio]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/api/datasets/marsyas/gtzan (Request ID: Root=1-66eeb0cc-2eb9159069eae9295cd93cc7;337b1461-8758-4663-8440-e6ea5e64e829)

Internal Error - We're working hard to fix this as soon as possible!

GTZAN doesn’t provide a predefined validation set, so we’ll have to create one ourselves. The dataset is balanced across genres, so we can use the train_test_split() method to quickly create a 90/10 split as follows:



In [ ]:
gtzan = gtzan['train'].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Great, now that we’ve got our training and validation sets, let’s take a look at one of the audio files:



In [ ]:
gtzan['train'][0]

As we saw in Unit 1, the audio files are represented as 1-dimensional NumPy arrays, where the value of the array represents the amplitude at that timestep. For these songs, the sampling rate is 22,050 Hz, meaning there are 22,050 amplitude values sampled per second. We’ll have to keep this in mind when using a pretrained model with a different sampling rate, converting the sampling rates ourselves to ensure they match. We can also see the genre is represented as an integer, or class label, which is the format the model will make it’s predictions in. Let’s use the int2str() method of the genre feature to map these integers to human-readable names:



In [ ]:
id2label_fn = gtzan['train'].features['genre'].int2str
id2label_fn(gtzan['train'][0]['genre'])

This label looks correct, since it matches the filename of the audio file. Let’s now listen to a few more examples by using Gradio to create a simple interface with the Blocks API:



In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def generate_audio():
  example = gtzan['train'].shuffle()[0]
  audio = example['audio']

  return (
      audio['sampling_rate'],
      audio['array']
  ), id2label_fn(example['genre'])

with gr.Blocks() as demo:
  with gr.Column():
    for _ in range(4):
      audio, label = generate_audio()
      output = gr.Audio(audio, label=label)

demo.launch(debug=True)

From these samples we can certainly hear the difference between genres, but can a transformer do this too? Let’s train a model to find out! First, we’ll need to find a suitable pretrained model for this task. Let’s see how we can do that.



## Picking a pretrained model for audio classification
To get started, let’s pick a suitable pretrained model for audio classification. In this domain, pretraining is typically carried out on large amounts of unlabeled audio data, using datasets like LibriSpeech and Voxpopuli. The best way to find these models on the Hugging Face Hub is to use the “Audio Classification” filter, as described in the previous section. Although models like Wav2Vec2 and HuBERT are very popular, we’ll use a model called DistilHuBERT. This is a much smaller (or distilled) version of the HuBERT model, which trains around 73% faster, yet preserves most of the performance.



### From audio to machine learning features
#### Preprocessing the data
Similar to tokenization in NLP, audio and speech models require the input to be encoded in a format that the model can process. In 🤗 Transformers, the conversion from audio to the input format is handled by the feature extractor of the model. Similar to tokenizers, 🤗 Transformers provides a convenient AutoFeatureExtractor class that can automatically select the correct feature extractor for a given model. To see how we can process our audio files, let’s begin by instantiating the feature extractor for DistilHuBERT from the pre-trained checkpoint:

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True,
    return_attention_mask=True
)

Since the sampling rate of the model and the dataset are different, we’ll have to resample the audio file to 16,000 Hz before passing it to the feature extractor. We can do this by first obtaining the model’s sample rate from the feature extractor:



In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

Next, we resample the dataset using the cast_column() method and Audio feature from 🤗 Datasets:



In [ ]:
from datasets import Audio

gtzan = gtzan.cast_column('audio', Audio(sampling_rate=sampling_rate))

We can now check the first sample of the train-split of our dataset to verify that it is indeed at 16,000 Hz. 🤗 Datasets will resample the audio file on-the-fly when we load each audio sample:



In [ ]:
gtzan["train"][0]

Great! We can see that the sampling rate has been downsampled to 16kHz. The array values are also different, as we’ve now only got approximately one amplitude value for every 1.5 that we had before.



A defining feature of Wav2Vec2 and HuBERT like models is that they accept a float array corresponding to the raw waveform of the speech signal as an input. This is in contrast to other models, like Whisper, where we pre-process the raw audio waveform to spectrogram format.

We mentioned that the audio data is represented as a 1-dimensional array, so it’s already in the right format to be read by the model (a set of continuous inputs at discrete time steps). So, what exactly does the feature extractor do?

Well, the audio data is in the right format, but we’ve imposed no restrictions on the values it can take. For our model to work optimally, we want to keep all the inputs within the same dynamic range. This is going to make sure we get a similar range of activations and gradients for our samples, helping with stability and convergence during training.

To do this, we normalise our audio data, by rescaling each sample to zero mean and unit variance, a process called feature scaling. It’s exactly this feature normalisation that our feature extractor performs!


In [ ]:
import numpy as np

sample = gtzan['train'][0]['audio']

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

We can see that the mean is close to zero already, but the variance is closer to 0.05. If the variance for the sample was larger, it could cause our model problems, since the dynamic range of the audio data would be very small and thus difficult to separate. Let’s apply the feature extractor and see what the outputs look like:



In [ ]:
inputs = feature_extractor(sample['array'], sampling_rate=sample['sampling_rate'])

print(f"inputs keys: {list(inputs.keys())}")
print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

We can see that the mean value is now very much closer to zero, and the variance bang-on one! This is exactly the form we want our audio samples in prior to feeding them to the HuBERT model.



Great, so now we know how to process our resampled audio files, the last thing to do is define a function that we can apply to all the examples in the dataset. Since we expect the audio clips to be 30 seconds in length, we’ll also truncate any longer clips by using the max_length and truncation arguments of the feature extractor as follows:



In [ ]:
max_duration = 30.0

def preprocess_function(examples):
  audio_arrays = [x['array'] for x in examples['audio']]
  inputs = feature_extractor(
      audio_arrays,
      sampling_rate=feature_extractor.sampling_rate,
      max_length=int(feature_extractor.sampling_rate * max_duration),
      truncation=True,
      return_attention_mask=True
  )

  return inputs

With this function defined, we can now apply it to the dataset using the map() method. The .map() method supports working with batches of examples, which we’ll enable by setting batched=True. The default batch size is 1000, but we’ll reduce it to 100 to ensure the peak RAM stays within a sensible range for Google Colab’s free tier:


In [ ]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=['audio','file'],
    batched=True,
    batch_size=100,
    num_proc=1
)

gtzan_encoded

To simplify the training, we’ve removed the audio and file columns from the dataset. The input_values column contains the encoded audio files, the attention_mask a binary mask of 0/1 values that indicate where we have padded the audio input, and the genre column contains the corresponding labels (or targets). To enable the Trainer to process the class labels, we need to rename the genre column to label:

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Finally, we need to obtain the label mappings from the dataset. This mapping will take us from integer ids (e.g. 7) to human-readable class labels (e.g. "pop") and back again. In doing so, we can convert our model’s integer id prediction into human-readable format, enabling us to use the model in any downstream application. We can do this by using the int2str() method as follows:



In [ ]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

OK, we’ve now got a dataset that’s ready for training! Let’s take a look at how we can train a model on this dataset.



## Fine-tuning the model


To fine-tune the model, we’ll use the Trainer class from 🤗 Transformers. As we’ve seen in other chapters, the Trainer is a high-level API that is designed to handle the most common training scenarios. In this case, we’ll use the Trainer to fine-tune the model on GTZAN. To do this, we’ll first need to load a model for this task. We can do this by using the AutoModelForAudioClassification class, which will automatically add the appropriate classification head to our pretrained DistilHuBERT model. Let’s go ahead and instantiate the model:

In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,

)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments


model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [ ]:
!pip install -q evaluate

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)